## 1. 模块库的引入

In [8]:
from langchain_community.document_loaders import PyPDFLoader  # PDF文档提取
from langchain_community.embeddings.self_hosted_hugging_face import load_embedding_model
from langchain_text_splitters import RecursiveCharacterTextSplitter  # 文档拆分chunk
from sentence_transformers import SentenceTransformer
import faiss  # faiss向量库
import numpy as np  # 处理嵌入向量数据，用于faiss向量检索
from http import HTTPStatus  # 检查与Qwen模型HTTP请求状态
import os  # 引入操作系统库，后续配置环境变量和获取文件路径
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # 不使用分词并行化操作，避免多线程或多进程环境中运行多个模型引发冲突

# 设置Qwen系列具体模型及对应的调用API密钥，从硅基流动大模型服务平台获得
chat_model = "Qwen/Qwen2.5-7B-Instruct"
embedding_model = "BAAI/bge-m3"
api_key = "sk-cfnxmdagbksbtroeforegefpyaawuuoijvwcmfbcvhgxzexb"
base_url = "https://api.siliconflow.cn/v1"

## 2. 索引流程

In [2]:
from openai import OpenAI

### 2.1 获取文本嵌入

In [ ]:
def load_embedding_model():
    """
    加载bge-m3模型
    :return: 返回bge-m3模型嵌入的结果
    """
    client = OpenAI(
            api_key=api_key, # 从https://cloud.siliconflow.cn/account/ak获取
            base_url=base_url
        )
    print(f"加载Embedding模型中")

    def get_embedding(text):
       text = text.replace("\n", " ")
       return client.embeddings.create(input = [text], model=embedding_model).data[0].embedding

    return get_embedding

### 2.2 建立索引，即把文本嵌入写入FAISS向量数据库

In [10]:
def indexing_process(pdf_file, embedding_model):
    """
    索引流程：加载PDF文件，并将其内容分割成小块，计算这些小块的嵌入向量并将其存储在FAISS向量数据库中。
    :param pdf_file: PDF文件路径
    :param embedding_model: 预加载的嵌入模型
    :return: 返回Faiss嵌入向量索引和分割后的文本块原始内容列表
    """
    # PyPDFLoader加载PDF文件，忽略图片提取
    pdf_loader = PyPDFLoader(pdf_file, extract_images=False)
    # 配置RecursiveCharacterTextSplitter分割文本块库参数，每个文本块的大小为512字符（非token），相邻文本块之间的重叠128字符（非token）
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512, chunk_overlap=128
    )
    # 加载PDF文档,提取所有页的文本内容
    pdf_content_list = pdf_loader.load()
    # 将每页的文本内容用换行符连接，合并为PDF文档的完整文本
    pdf_text = "\n".join([page.page_content for page in pdf_content_list])
    print(f"PDF文档的总字符数: {len(pdf_text)}")

    # 将PDF文档文本分割成文本块Chunk
    chunks = text_splitter.split_text(pdf_text)
    print(f"分割的文本Chunk数量: {len(chunks)}")

    # 文本块转化为嵌入向量列表，normalize_embeddings表示对嵌入向量进行归一化，用于准确计算相似度
    embeddings = []
    for chunk in chunks:
        embedding = embedding_model(chunk)
        embeddings.append(embedding)

    print("文本块Chunk转化为嵌入向量完成")

    # 将嵌入向量列表转化为numpy数组，Faiss向量库操作需要numpy数组输入
    embeddings_np = np.array(embeddings)

    # 获取嵌入向量的维度（每个向量的长度）
    dimension = embeddings_np.shape[1]

    # 使用余弦相似度创建FAISS索引
    index = faiss.IndexFlatIP(dimension)
    # 将所有的嵌入向量添加到FAISS索引中，后续可以用来进行相似性检索
    index.add(embeddings_np)

    print("索引过程完成.")

    return index, chunks

In [13]:
def main():
    print("RAG过程开始.")

    embedding_model = load_embedding_model()
    index, chunks = indexing_process('data/lesson1.pdf', embedding_model)
    print(index)
    print(chunks)


In [14]:
main()

RAG过程开始.
加载Embedding模型中
PDF文档的总字符数: 9163
分割的文本Chunk数量: 24
文本块Chunk转化为嵌入向量完成
索引过程完成.
<faiss.swigfaiss_avx2.IndexFlatIP; proxy of <Swig Object of type 'faiss::IndexFlatIP *' at 0x0000018C585C7ED0> >
['数字化转型\n1. 数字化转型的背景和意义\n1.1 背景\n在过去的十年里，数字技术的迅猛发展已彻底改变了企业运营的方方面面。互联网、移动技\n术、云计算、大数据、物联网（IoT）以及人工智能（AI）等技术的广泛应用，不仅催生了\n新的商业模式，还推动了企业在各个层面上的深度变革。企业开始意识到，数字化转型不仅\n仅是技术的应用，更是企业生存和发展的必然选择。\n1.2 意义\n1.2.1 提升企业竞争力\n数字化转型帮助企业在各个方面优化流程、提高效率并降低成本，使其能够在全球市场\n中保持竞争优势。通过对核心业务流程的自动化改造，企业能够减少人工操作的依赖，显著\n提升工作效率和生产力。例如，借助机器人流程自动化（RPA）技术，企业可以将繁琐的审\n批和处理流程缩短为更高效的操作，从而节省时间和成本。\n此外，数字化技术使企业能够实现数据驱动的决策。通过引入人工智能和大数据分析工\n具，企业可以深入分析客户行为、市场趋势和运营数据，从而做出更为精准的商业决策。这\n样的数据驱动策略不仅提升了企业的竞争力，还增强了其对市场变化的敏感性，使其能够快\n速调整策略以应对新的挑战和机遇。', '具，企业可以深入分析客户行为、市场趋势和运营数据，从而做出更为精准的商业决策。这\n样的数据驱动策略不仅提升了企业的竞争力，还增强了其对市场变化的敏感性，使其能够快\n速调整策略以应对新的挑战和机遇。\n最后，数字化转型提升了企业的创新能力。借助数字化平台和技术，企业可以更快地开\n发和推出新产品和服务，满足消费者不断变化的需求。这种创新能力使企业能够在激烈的市\n场竞争中脱颖而出，保持持续增长。同时，数字化转型还为企业开辟了新的市场和业务模式，\n进一步巩固了其在全球市场中的地位。\n1.2.2 改善客户体验\n现代客户的期望已经发生了显著变化，他们不再满足于传统的一刀切式的服务模式，